In [1]:
# tutorial for finetuning whisper
# https://huggingface.co/blog/fine-tune-whisper

In [2]:
import torch
print("MPS:", torch.backends.mps.is_available())

import sys
print(sys.executable)

MPS: True
/Users/zuzamakowska/Documents/Africa/Project/Low-resource-languages/venv/bin/python


In [3]:
# huggingface-cli whoami <- to check if you're logged in to hugging face 

## Download Common Voice dataset (Swahili)

In [4]:
from datasets.utils.logging import set_verbosity_info
set_verbosity_info()

/Users/zuzamakowska/Documents/Africa/Project/Low-resource-languages/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# api key: echo $MDC_API_KEY

from datasets import load_dataset, Features, Value, Audio

features = Features({
    "client_id": Value("string"),
    "path": Value("string"),
    "sentence_id": Value("string"),
    "sentence": Value("string"),
    "sentence_domain": Value("string"),
    "up_votes": Value("string"),
    "down_votes": Value("string"),
    "age": Value("string"),
    "gender": Value("string"),
    "accents": Value("string"),
    "variant": Value("string"),
    "locale": Value("string"),
    "segment": Value("string"),
})

ds = load_dataset(
    "csv",
    data_files={
        "train": "../../data/cv-corpus-23.0-2025-09-05/sw/train.tsv",
        "validation": "../../data/cv-corpus-23.0-2025-09-05/sw/dev.tsv",
        "test": "../../data/cv-corpus-23.0-2025-09-05/sw/test.tsv"
    },
    delimiter="\t",
    features=features,
)


Using custom data configuration default-6acbb9350cfbc4a1
Generating dataset csv (/Users/zuzamakowska/.cache/huggingface/datasets/csv/default-6acbb9350cfbc4a1/0.0.0/a43390c7ecea6519ff2ce9d10005c8750601c9e456069be5efbd2747df45f420)
Checksum Computation took 0.0 min
Generating train split
Generating train split: 46611 examples [00:00, 311051.28 examples/s]
Generating validation split
Generating validation split: 11692 examples [00:00, 294471.45 examples/s]
Generating test split
Generating test split: 11944 examples [00:00, 302730.59 examples/s]
Unable to verify splits sizes.
Dataset csv downloaded and prepared to /Users/zuzamakowska/.cache/huggingface/datasets/csv/default-6acbb9350cfbc4a1/0.0.0/a43390c7ecea6519ff2ce9d10005c8750601c9e456069be5efbd2747df45f420. Subsequent calls will reuse this data.


In [6]:
print(ds["train"][0])
print(ds["train"].features)

{'client_id': 'f0e3f1cbc3526ea2273da567cb59583344ad1606b7be2ad13a902e8b971af62a3da34c20d4ee0e9ee59812079b0400ef7d2091514e02e74fa44c27a2c9ca5862', 'path': 'common_voice_sw_30558307.mp3', 'sentence_id': '5071bd0e179a4bb6d66e567969bcedb020858ddde6ed0eda07174e909497afe0', 'sentence': 'deLima alifunga mabao mawili kwenye fainali ya kombe la dunia', 'sentence_domain': None, 'up_votes': '2', 'down_votes': '0', 'age': 'thirties', 'gender': 'female_feminine', 'accents': None, 'variant': None, 'locale': 'sw', 'segment': None}
{'client_id': Value('string'), 'path': Value('string'), 'sentence_id': Value('string'), 'sentence': Value('string'), 'sentence_domain': Value('string'), 'up_votes': Value('string'), 'down_votes': Value('string'), 'age': Value('string'), 'gender': Value('string'), 'accents': Value('string'), 'variant': Value('string'), 'locale': Value('string'), 'segment': Value('string')}


In [7]:
def fix_path(batch):
    batch["path"] = "/Users/zuzamakowska/Documents/Africa/Project/Low-resource-languages/data/cv-corpus-23.0-2025-09-05/sw/clips/" + batch["path"]
    return batch

ds = ds.map(fix_path)
print(ds["train"].features)

Map: 100%|██████████| 11944/11944 [00:00<00:00, 19650.00 examples/s]

{'client_id': Value('string'), 'path': Value('string'), 'sentence_id': Value('string'), 'sentence': Value('string'), 'sentence_domain': Value('string'), 'up_votes': Value('string'), 'down_votes': Value('string'), 'age': Value('string'), 'gender': Value('string'), 'accents': Value('string'), 'variant': Value('string'), 'locale': Value('string'), 'segment': Value('string')}


In [8]:
from datasets import Audio
ds = ds.cast_column("path", Audio(sampling_rate=16000))

In [9]:

print(ds["train"].features)
print(ds["train"][0])


{'client_id': Value('string'), 'path': Audio(sampling_rate=16000, decode=True, num_channels=None, stream_index=None), 'sentence_id': Value('string'), 'sentence': Value('string'), 'sentence_domain': Value('string'), 'up_votes': Value('string'), 'down_votes': Value('string'), 'age': Value('string'), 'gender': Value('string'), 'accents': Value('string'), 'variant': Value('string'), 'locale': Value('string'), 'segment': Value('string')}
{'client_id': 'f0e3f1cbc3526ea2273da567cb59583344ad1606b7be2ad13a902e8b971af62a3da34c20d4ee0e9ee59812079b0400ef7d2091514e02e74fa44c27a2c9ca5862', 'path': <datasets.features._torchcodec.AudioDecoder object at 0x15078a230>, 'sentence_id': '5071bd0e179a4bb6d66e567969bcedb020858ddde6ed0eda07174e909497afe0', 'sentence': 'deLima alifunga mabao mawili kwenye fainali ya kombe la dunia', 'sentence_domain': None, 'up_votes': '2', 'down_votes': '0', 'age': 'thirties', 'gender': 'female_feminine', 'accents': None, 'variant': None, 'locale': 'sw', 'segment': None}


In [10]:
ds = ds.remove_columns(['client_id', 'sentence_id', 'sentence_domain', 'up_votes', 'down_votes', 'age', 'gender', 'accents', 'locale', 'segment'])

In [11]:
ds

DatasetDict({
    train: Dataset({
        features: ['path', 'sentence', 'variant'],
        num_rows: 46611
    })
    validation: Dataset({
        features: ['path', 'sentence', 'variant'],
        num_rows: 11692
    })
    test: Dataset({
        features: ['path', 'sentence', 'variant'],
        num_rows: 11944
    })
})

In [12]:
print(ds['train'].column_names)
print(ds['train'].features)

['path', 'sentence', 'variant']
{'path': Audio(sampling_rate=16000, decode=True, num_channels=None, stream_index=None), 'sentence': Value('string'), 'variant': Value('string')}


In [13]:
print(ds["train"][0])

{'path': <datasets.features._torchcodec.AudioDecoder object at 0x1514f3850>, 'sentence': 'deLima alifunga mabao mawili kwenye fainali ya kombe la dunia', 'variant': None}


In [14]:
ds = ds.with_format("numpy")
ds = ds.rename_column("path", "audio")
sample = ds["train"][0]

In [15]:
print(sample["audio"])

In [16]:
print(ds['train'][0])

{'audio': <datasets.features._torchcodec.AudioDecoder object at 0x178cdbeb0>, 'sentence': np.str_('deLima alifunga mabao mawili kwenye fainali ya kombe la dunia'), 'variant': None}


## Features Extraction

In [17]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="Swahili", task="transcribe", padding='longest')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained('openai/whisper-tiny')

In [19]:
input_str = ds['train'][0]['sentence']
# labels = tokenizer(input_str).input_ids
input_str

np.str_('deLima alifunga mabao mawili kwenye fainali ya kombe la dunia')

In [20]:
labels = tokenizer(input_str).input_ids
labels

[50258,
 50318,
 50359,
 50363,
 1479,
 43,
 4775,
 419,
 351,
 1063,
 64,
 275,
 5509,
 78,
 463,
 86,
 2312,
 350,
 15615,
 1200,
 283,
 491,
 5103,
 2478,
 5207,
 650,
 635,
 10234,
 654,
 50257]

In [21]:
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_with_special

'<|startoftranscript|><|sw|><|transcribe|><|notimestamps|>deLima alifunga mabao mawili kwenye fainali ya kombe la dunia<|endoftext|>'

In [22]:
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)
decoded_str

'deLima alifunga mabao mawili kwenye fainali ya kombe la dunia'

In [23]:
raw_tokens = tokenizer(input_str)
raw_tokens

{'input_ids': [50258, 50318, 50359, 50363, 1479, 43, 4775, 419, 351, 1063, 64, 275, 5509, 78, 463, 86, 2312, 350, 15615, 1200, 283, 491, 5103, 2478, 5207, 650, 635, 10234, 654, 50257], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [24]:
decoded_tokens = tokenizer.convert_ids_to_tokens(labels)
print(decoded_tokens)

['<|startoftranscript|>', '<|sw|>', '<|transcribe|>', '<|notimestamps|>', 'de', 'L', 'ima', 'Ġal', 'if', 'ung', 'a', 'Ġm', 'aba', 'o', 'Ġma', 'w', 'ili', 'Ġk', 'wen', 'ye', 'Ġf', 'ain', 'ali', 'Ġya', 'Ġkom', 'be', 'Ġla', 'Ġdun', 'ia', '<|endoftext|>']


### WhisperProcessor

In [25]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained('openai/whisper-tiny', language='Swahili', task='transcribe')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [26]:
print(ds["train"][0])

{'audio': <datasets.features._torchcodec.AudioDecoder object at 0x17da57e20>, 'sentence': np.str_('deLima alifunga mabao mawili kwenye fainali ya kombe la dunia'), 'variant': None}


In [27]:
from datasets import Audio

ds = ds.cast_column('audio', Audio(sampling_rate=16000))

In [28]:
print(ds["train"][0])

{'audio': <datasets.features._torchcodec.AudioDecoder object at 0x1791f84f0>, 'sentence': np.str_('deLima alifunga mabao mawili kwenye fainali ya kombe la dunia'), 'variant': None}


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate = audio["sampling_rate"]).input_features[0]
    batch["labels"] = tokenizer(batch["sentence"], ).input_ids
    return batch

In [ ]:
preprocessed_ds = ds.map(prepare_dataset, num_proc=4)

Process #1 will write at /Users/zuzamakowska/.cache/huggingface/datasets/csv/default-6acbb9350cfbc4a1/0.0.0/a43390c7ecea6519ff2ce9d10005c8750601c9e456069be5efbd2747df45f420/cache-64b1bfbdf990e579_00000_of_00004.arrow
Process #2 will write at /Users/zuzamakowska/.cache/huggingface/datasets/csv/default-6acbb9350cfbc4a1/0.0.0/a43390c7ecea6519ff2ce9d10005c8750601c9e456069be5efbd2747df45f420/cache-64b1bfbdf990e579_00001_of_00004.arrow
Process #3 will write at /Users/zuzamakowska/.cache/huggingface/datasets/csv/default-6acbb9350cfbc4a1/0.0.0/a43390c7ecea6519ff2ce9d10005c8750601c9e456069be5efbd2747df45f420/cache-64b1bfbdf990e579_00002_of_00004.arrow
Process #4 will write at /Users/zuzamakowska/.cache/huggingface/datasets/csv/default-6acbb9350cfbc4a1/0.0.0/a43390c7ecea6519ff2ce9d10005c8750601c9e456069be5efbd2747df45f420/cache-64b1bfbdf990e579_00003_of_00004.arrow
Map (num_proc=4):   0%|          | 0/46611 [00:00<?, ? examples/s]Spawning 4 processes
Caching processed dataset at /Users/zuzamakow

In [31]:
preprocessed_ds

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence', 'variant', 'input_features', 'labels'],
        num_rows: 46611
    })
    validation: Dataset({
        features: ['audio', 'sentence', 'variant', 'input_features', 'labels'],
        num_rows: 11692
    })
    test: Dataset({
        features: ['audio', 'sentence', 'variant', 'input_features', 'labels'],
        num_rows: 11944
    })
})

In [32]:
from transformers import WhisperForConditionalGeneration
model = WhisperForConditionalGeneration.from_pretrained('openai/whisper-tiny')


In [33]:
model.generation_config.language = "swahili"
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None

In [34]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [35]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [36]:
import evaluate

wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

In [37]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * wer_metric.compute(predictions=pred_str, references=label_str)
    cer = 100 * cer_metric.compute(predictions=pred_str, references=label_str)

    return {
        "wer": wer,
        "cer": cer,
        "combined": 0.5 * wer + 0.5 * cer,
    }

In [38]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="../../models/whisper-tiny-sw",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=False,
    evaluation_strategy="steps",
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="combined",
    greater_is_better=False,
    push_to_hub=False,
    logging_strategy="steps",
)


/Users/zuzamakowska/Documents/Africa/Project/Low-resource-languages/venv/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [39]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=preprocessed_ds["train"],
    eval_dataset=preprocessed_ds["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


max_steps is given, it will override any value given in num_train_epochs


In [40]:
trainer.train()

  0%|          | 0/5000 [00:00<?, ?it/s]/Users/zuzamakowska/Documents/Africa/Project/Low-resource-languages/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/zuzamakowska/Documents/Africa/Project/Low-resource-languages/venv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
  0%|          | 25/5000 [00:38<

{'loss': 3.2518, 'grad_norm': 37.26277160644531, 'learning_rate': 5.000000000000001e-07, 'epoch': 0.01}


  1%|          | 50/5000 [01:16<2:02:58,  1.49s/it]

{'loss': 2.884, 'grad_norm': 31.137889862060547, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.02}


  2%|▏         | 75/5000 [01:53<2:02:05,  1.49s/it]

...

 20%|█▉        | 975/5000 [24:10<1:39:13,  1.48s/it]

{'loss': 0.864, 'grad_norm': 14.82085132598877, 'learning_rate': 8.944444444444446e-06, 'epoch': 0.33}


 20%|██        | 1000/5000 [24:47<1:37:59,  1.47s/it]

{'loss': 0.9056, 'grad_norm': 16.731088638305664, 'learning_rate': 8.888888888888888e-06, 'epoch': 0.34}


RuntimeError: The size of tensor a (2503) must match the size of tensor b (448) at non-singleton dimension 1

In [47]:
print("max_source_positions:", model.config.max_source_positions)
print("max_target_positions:", model.config.max_target_positions)
print("max_length:", model.config.max_length)

max_source_positions: 1500
max_target_positions: 448
max_length: 448


In [49]:
def add_label_length(batch):
    batch["label_length"] = len(batch["labels"])
    batch["audio_length"] = batch["input_features"].shape[-1]  # liczba ramek melspektrogramu
    return batch

preprocessed_ds = preprocessed_ds.map(add_label_length)

max_label_len = model.config.max_target_positions     # np. 448
max_audio_len = model.config.max_source_positions     # np. 1500 dla whisper-tiny

for split in preprocessed_ds.keys():
    labels = preprocessed_ds[split]["label_length"]
    audio = preprocessed_ds[split]["audio_length"]

    total = len(labels)

    too_long_labels = sum(l > max_label_len for l in labels)
    too_long_audio = sum(a > max_audio_len for a in audio)

    print(f"\n=== {split} ===")
    print(f"- Za długie transkrypcje: {too_long_labels}/{total} (>{max_label_len} tokenów)")
    print(f"- Za długie audio:        {too_long_audio}/{total} (>{max_audio_len} frames)")
    print(f"- max długość labeli:  {max(labels)}")
    print(f"- max długość audio:   {max(audio)}")



Caching processed dataset at /Users/zuzamakowska/.cache/huggingface/datasets/csv/default-6acbb9350cfbc4a1/0.0.0/a43390c7ecea6519ff2ce9d10005c8750601c9e456069be5efbd2747df45f420/cache-19cc6a6653ddbe93.arrow




























































































































































































































































































































































































































































































































































































































































































































































































































=== train ===
- Za długie transkrypcje: 1/46611 (>448 tokenów)
- Za długie audio:        46611/46611 (>1500 frames)
- max długość labeli:  1103
- max długość audio:   3000

=== validation ===
- Za długie transkrypcje: 2/11692 (>448 tokenów)
- Za długie audio:        11692/11692 (>1500 frames)
- max długość labeli:  82023
- max długość audio:   3000

=== test ===
- Za długie transkrypcje: 3/11944 (>448 tokenów)
- Za długie audio:        11944/11944 (>1500 frames)
- max długość labeli:  48420
- max długość audio:   3000


48420

3000